In [2]:
import os
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

from emnist import list_datasets, extract_training_samples, extract_test_samples

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import tensorflow as tf
from tensorflow import keras

def flatten(data, dim_img = (28, 28)):
    return data.reshape(len(data), dim_img[0] * dim_img[1])
    
print(list_datasets())

['balanced', 'byclass', 'bymerge', 'digits', 'letters', 'mnist']


In [4]:
tipus = "balanced"

X_full_train, y_full_train = extract_training_samples(tipus)
X_test, y_test = extract_test_samples(tipus)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size = 0.1, random_state = 123)

In [5]:
import joblib
# save models

SVC_model = joblib.load("models/svc_model2.joblib")
# RF_model = joblib.load("models/RF_model1.joblib")
ANN_model = keras.models.load_model("models/nn_model.h5")
CNN_model = keras.models.load_model("models/cnn_model.h5")

In [8]:
print(SVC_model.score(flatten(X_val), y_val))
print(SVC_model.score(flatten(X_test), y_test))

0.8020390070921986
0.798936170212766


In [10]:
print(ANN_model.evaluate(X_val / 255, y_val))
print(ANN_model.evaluate(X_test / 255, y_test))

353/353 [==============================] - 2s 1ms/step - loss: 0.4708 - accuracy: 0.8485
[0.4707845449447632, 0.8484929203987122]
588/588 [==============================] - 1s 2ms/step - loss: 0.4996 - accuracy: 0.8395
[0.4996272921562195, 0.8394680619239807]


In [11]:
print(CNN_model.evaluate(X_val[..., np.newaxis] / 255, y_val))
print(CNN_model.evaluate(X_test[..., np.newaxis] / 255, y_test))

353/353 [==============================] - 4s 11ms/step - loss: 0.3361 - accuracy: 0.8805
[0.33608120679855347, 0.8804964423179626]
588/588 [==============================] - 7s 11ms/step - loss: 0.3715 - accuracy: 0.8745
[0.37146854400634766, 0.8744680881500244]


In [ ]:
def label_to_letter(label, tipus = "byclass"):
    if tipus == "byclass" or tipus == "mnist" or tipus == "digits":        
        if label >= 0 and label <= 9:
            return label
        elif label >= 10 and label <= 35:
            return chr(label + 55)
        elif label >= 36 and label <= 61:
            return chr(label + 61)
        else:
            raise ValueError("Label '{}' fora dels limits".format(label))

    elif tipus == "bymerge" or tipus == "balanced":
        bymerge_dic = [
            "0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
            "A", "B", "C/c", "D", "E", "F", "G", "H", "I/i", "J/j", 
            "K/k", "L/l", "M/m", "N", "O/o", "P/p", "Q", "R", "S/s", "T", 
            "U/u", "V/v", "W/w", "X/x", "Y/y", "Z/z", "a", "b", "d", "e", 
            "f", "g", "h", "n", "q", "r", "t"]
        return bymerge_dic[label]

    elif tipus == "letters":
        return chr(label + 55 + 10 - 1)

label_to_letter(1)

In [15]:
y_pred = ANN_model.predict(X_val, ) 

In [ ]:
["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", 
"A", "B", "C/c", "D", "E", "F", "G", "H", "I/i", "J/j", 
"K/k", "L/l", "M/m", "N", "O/o", "P/p", "Q", "R", "S/s", "T", 
"U/u", "V/v", "W/w", "X/x", "Y/y", "Z/z", "a", "b", "d", "e", 
"f", "g", "h", "n", "q", "r", "t"]

In [16]:
y_pred.shape

(11280, 47)

In [ ]:
def plot_conf(matrix, figsize = (16, 16)):
    fig = plt.figure(figsize = figsize)
    ax = fig.add_subplot(111)
    cax = ax.matshow(matrix)
    fig.colorbar(cax)

row_sums = conf_matrx.sum(axis = 1, keepdims = True)
norm_conf_matrx = conf_matrx / row_sums
np.fill_diagonal(norm_conf_matrx, 0)
plot_conf(norm_conf_matrx)